In [1]:
# May 2019, showed that neural network predistortion approach is valid (pure python)
# June 2019, showed that we can do calibration using the real data from Sungjae (scikit learn and tensorflow)
# Sept. 2019, now I switched to pytorch, I am going to implement really simple NN predistorter

### Training and test samples generation

In [2]:
import math
from torch.utils.data import DataLoader
import torch

QAM_size = 2**14
num_interval = math.sqrt(QAM_size) - 1
interval_size = 2 / num_interval

theta_00 = 0.01
theta_01 = 0.98  # close to 1
theta_02 = 0.02
theta_03 = 0.01
theta_04 = interval_size / 5

theta_10 = 0.01
theta_11 = 0.02
theta_12 = 0.01
theta_13 = 1.01  # close to 1
theta_14 = interval_size / 5

numTrainSamples = 10000
numTestSamples = 1000

error_bound = interval_size / 20


# 미니배치크기
BATCH = 10
# 디바이스 설정: cuda 사용할지 cpu 사용할지
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# 총 스텝 크기
print(DEVICE)
STEP = 20000
PRINT_STEP = 100

INPUT_SIZE = 2
HIDDEN_SIZE_0 = 10
OUTPUT_SIZE = 2

model_name = "predistortion_lr5_step20000"

cuda


In [3]:
# Quadratic relation formulation
# y_0 = theta_00 * x_0**2 + theta_01 * x_0 + theta_02 * x_1**2 + theta_03 * x_1 + theta_04
# y_1 = theta_10 * x_0**2 + theta_11 * x_0 + theta_12 * x_1**2 + theta_13 * x_1 + theta_14
# theta_01 and theta_13 will probably be ~= 1 and other coefficients should be ~= 0
# training_data is a np.array of tuples

import numpy as np
import random
random.seed(42)

I_range = []
for counter in range(int(num_interval + 1)):
    I_value = -1 + interval_size * counter  # range is from -1 to 1
    I_range = I_range + [I_value]
Q_range = I_range


# Data generation
trainDataInputs = []
testDataInputs = []
counter = 0
while(counter < (numTrainSamples+numTestSamples)):
    I = random.choice(I_range)
    Q = random.choice(Q_range)
    if((I, Q) not in trainDataInputs and (I, Q) not in testDataInputs):
        if(counter < numTrainSamples):
            trainDataInputs = trainDataInputs + [(I, Q)]
        else:
            testDataInputs = testDataInputs + [(I, Q)]
        counter = counter + 1


trainDataList = []
testDataList = []

for (I_in, Q_in) in trainDataInputs:
    I_out = theta_00 * I_in**2 + theta_01 * I_in + theta_02 * Q_in**2 + theta_03 * Q_in + theta_04
    Q_out = theta_10 * I_in**2 + theta_11 * I_in + theta_12 * Q_in**2 + theta_13 * Q_in + theta_14

    # predistorter's (input_variable,output_variable)
    # AND... THIS WAS DONE ALREADY. NICE
    data = (torch.tensor([I_out,Q_out]),torch.tensor([I_in,Q_in]))
    trainDataList = trainDataList + [data]

for (I_in, Q_in) in testDataInputs:
    I_out = theta_00 * I_in**2 + theta_01 * I_in + theta_02 * Q_in**2 + theta_03 * Q_in + theta_04
    Q_out = theta_10 * I_in**2 + theta_11 * I_in + theta_12 * Q_in**2 + theta_13 * Q_in + theta_14

    # predistorter's (input_variable,output_variable)
    data = (torch.tensor([I_out,Q_out]),torch.tensor([I_in,Q_in]))
    testDataList = testDataList + [data]
    
    
print(testDataList[:5])
print(testDataList[0])
print(testDataList[0][0])
print(testDataList[0][1])

print(len(trainDataList))
print(len(testDataList))



[(tensor([-0.7807, -0.7877]), tensor([-0.8110, -0.7795])), (tensor([0.4219, 0.4525]), tensor([0.4173, 0.4331])), (tensor([-0.9487,  0.2083]), tensor([-0.9843,  0.2126])), (tensor([-0.6109, -0.8963]), tensor([-0.6378, -0.8898])), (tensor([ 0.1798, -0.1913]), tensor([ 0.1811, -0.1969]))]
(tensor([-0.7807, -0.7877]), tensor([-0.8110, -0.7795]))
tensor([-0.7807, -0.7877])
tensor([-0.8110, -0.7795])
10000
1000


In [4]:
# 훈련 데이터로더 선언
train_loader = DataLoader(dataset=testDataList, 
                          batch_size=BATCH, 
                          shuffle=True)
# 테스트 데이터 로더 설정
test_loader = DataLoader(dataset=testDataList, 
                         batch_size=BATCH, 
                         shuffle=True)

In [5]:
for (data, target) in train_loader:
    print(data.size(), target.size())
    break

torch.Size([10, 2]) torch.Size([10, 2])


### Pytorch Neural Network generation

In [6]:
import torch.nn as nn
import torch.optim as optim


In [7]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size_0, output_size):
        super(Net, self).__init__()
        self.flatten = lambda x: x.view(x.size(0), -1)
        self.linear1 = nn.Linear(input_size, hidden_size_0)
        self.linear2 = nn.Linear(hidden_size_0, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.tanh(x)        
        x = self.linear2(x)
        return x

In [8]:
model = Net(input_size=INPUT_SIZE, hidden_size_0=HIDDEN_SIZE_0, output_size=OUTPUT_SIZE).to(DEVICE)

In [9]:
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.00005)

# 매개변수 개수 확인하기
num_params = 0
for params in model.parameters():
    num_params += params.view(-1).size(0)
print("Total number of parameters: {}".format(num_params))

Total number of parameters: 52


In [10]:
def train(model, train_loader, loss_func, optimizer, step, device, print_step=50):
    """train function: 1 스텝 동안 발생하는 학습과정"""
    # 모델에게 훈련단계이라고 선언함
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 입력과 타겟 텐서에 GPU 를 사용여부 전달
        data, target = data.to(device), target.to(device)
        # 경사 초기화
        model.zero_grad()
        # 순방향 전파
        output = model(data)
        # 손실값 계산
        loss = loss_func(output, target)
        # 역방향 전파
        loss.backward()
        # 매개변수 업데이트
        optimizer.step()


    if(step%print_step==0):
        print('Train Step: {}  \tLoss: {:.8f}'.format(
                step,  loss.item()))
def test(model, test_loader, loss_func, device, step, print_step=50):
    """test function"""
    # 모델에게 평가단계이라고 선언함
    model.eval()
    test_loss = 0
    correct = 0

    sum_score = 0
#     total_score = 0
    with torch.no_grad():
        for data, target in test_loader:
            # 입력과 타겟 텐서에 GPU 를 사용여부 전달
            data, target = data.to(device), target.to(device)
            # 순방향전파
            output = model(data)            

            score_this_batch = 0
            diff_tensor = abs(target-output)
            for i in range(test_loader.batch_size):
                if(diff_tensor[i][0]<error_bound and diff_tensor[i][1]<error_bound):
                    score_this_batch = score_this_batch + 1
            sum_score = sum_score + score_this_batch

    acc = sum_score / len(test_loader.dataset)
            
#     test_loss /= len(test_loader.dataset)
#     test_acc = correct / len(test_loader.dataset)
    if(step%print_step==0):
        print('Test set: Accuracy: {}/{} ({:05.2f}%)'.format(
            sum_score, len(test_loader.dataset), 100. * acc))
    return sum_score, acc

def main(model, train_loader, test_loader, loss_func, optimizer, n_step, device, save_path=None, print_step=50):
    """메인 학습 함수"""
    test_score_list = []

    for step in range(1, n_step+1):
        # 훈련 단계
        train(model, train_loader, loss_func, optimizer, 
              step=step, device=device, print_step=print_step)
        # 평가 단계
        sum_score, acc = test(model, test_loader, loss_func, device=device, step=step, print_step=print_step)
        test_score_list = test_score_list + [sum_score]

    print(test_score_list)
    torch.save(model.state_dict(), save_path)

In [11]:
main(model=model, 
     train_loader=train_loader, 
     test_loader=test_loader, 
     loss_func=loss_function, 
     optimizer=optimizer, 
     n_step=STEP,
     device=DEVICE,
     save_path=model_name+".pt", 
     print_step=PRINT_STEP)

Train Step: 100  	Loss: 0.00153898
Test set: Accuracy: 0/1000 (00.00%)
Train Step: 200  	Loss: 0.00049156
Test set: Accuracy: 1/1000 (00.10%)
Train Step: 300  	Loss: 0.00060459
Test set: Accuracy: 1/1000 (00.10%)
Train Step: 400  	Loss: 0.00014566
Test set: Accuracy: 3/1000 (00.30%)
Train Step: 500  	Loss: 0.00013915
Test set: Accuracy: 2/1000 (00.20%)
Train Step: 600  	Loss: 0.00017948
Test set: Accuracy: 8/1000 (00.80%)
Train Step: 700  	Loss: 0.00034954
Test set: Accuracy: 9/1000 (00.90%)
Train Step: 800  	Loss: 0.00010958
Test set: Accuracy: 8/1000 (00.80%)
Train Step: 900  	Loss: 0.00006286
Test set: Accuracy: 5/1000 (00.50%)
Train Step: 1000  	Loss: 0.00008174
Test set: Accuracy: 2/1000 (00.20%)
Train Step: 1100  	Loss: 0.00008224
Test set: Accuracy: 5/1000 (00.50%)
Train Step: 1200  	Loss: 0.00002699
Test set: Accuracy: 8/1000 (00.80%)
Train Step: 1300  	Loss: 0.00003125
Test set: Accuracy: 8/1000 (00.80%)
Train Step: 1400  	Loss: 0.00002237
Test set: Accuracy: 7/1000 (00.70%)
T

Train Step: 11200  	Loss: 0.00000007
Test set: Accuracy: 959/1000 (95.90%)
Train Step: 11300  	Loss: 0.00000010
Test set: Accuracy: 939/1000 (93.90%)
Train Step: 11400  	Loss: 0.00000008
Test set: Accuracy: 980/1000 (98.00%)
Train Step: 11500  	Loss: 0.00000013
Test set: Accuracy: 955/1000 (95.50%)
Train Step: 11600  	Loss: 0.00000012
Test set: Accuracy: 958/1000 (95.80%)
Train Step: 11700  	Loss: 0.00000011
Test set: Accuracy: 982/1000 (98.20%)
Train Step: 11800  	Loss: 0.00000006
Test set: Accuracy: 963/1000 (96.30%)
Train Step: 11900  	Loss: 0.00000012
Test set: Accuracy: 986/1000 (98.60%)
Train Step: 12000  	Loss: 0.00000007
Test set: Accuracy: 978/1000 (97.80%)
Train Step: 12100  	Loss: 0.00000005
Test set: Accuracy: 970/1000 (97.00%)
Train Step: 12200  	Loss: 0.00000017
Test set: Accuracy: 986/1000 (98.60%)
Train Step: 12300  	Loss: 0.00000008
Test set: Accuracy: 985/1000 (98.50%)
Train Step: 12400  	Loss: 0.00000005
Test set: Accuracy: 984/1000 (98.40%)
Train Step: 12500  	Loss:

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 2, 2, 1, 2, 3, 4, 1, 3, 2, 2, 1, 2, 1, 1, 0, 1, 2, 1, 1, 0, 1, 0, 3, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 